In [3]:
# imports

import pandas as pd
import requests
from dotenv import load_dotenv
import os
import numpy as np
import json




# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [4]:
# Access API Keys
load_dotenv(dotenv_path='/Users/mitchellpalmer/Projects/Lighthouse Lab Projects/Data_Statistical_Modeling/Statistical_Modeling_Project/Statistical-Modelling-Project./data/.env')
api_key = os.getenv('FOURSQUARE_Project_API_KEY')


In [5]:
# Upload Lisbon CityBike Data .CSV
lb_citybikes = pd.read_csv('/Users/mitchellpalmer/Projects/Lighthouse Lab Projects/Data_Statistical_Modeling/Statistical_Modeling_Project/Statistical-Modelling-Project./data/lisbon_bike_station_df.csv')

lb_citybikes

,latitude,longitude,timestamp,free bikes,empty slots,total slots,station_id
0,38.753590,-9.157170,2025-07-26T23:39:44.571202+00:00Z,0,32,32,0
1,38.752310,-9.158850,2025-07-26T23:39:44.571444+00:00Z,0,23,23,1
2,38.770657,-9.160248,2025-07-26T23:39:44.570809+00:00Z,7,26,33,2
3,38.780690,-9.096220,2025-07-26T23:39:44.570706+00:00Z,8,6,14,3
4,38.702936,-9.175234,2025-07-26T23:39:44.571010+00:00Z,6,17,23,4
...,...,...,...,...,...,...,...
190,38.729724,-9.157531,2025-07-26T23:39:44.571251+00:00Z,12,18,30,190
191,38.711498,-9.194361,2025-07-26T23:39:44.570595+00:00Z,1,40,41,191
192,38.739395,-9.162447,2025-07-26T23:39:44.571255+00:00Z,0,17,17,192
193,38.708780,-9.137125,2025-07-26T23:39:44.571246+00:00Z,16,4,20,193


In [6]:
# Parameters
lb_citybikes['coordinates'] = lb_citybikes['latitude'].astype(str) + ',' + lb_citybikes['longitude'].astype(str)


lb_citybikes.head(5)

,latitude,longitude,timestamp,free bikes,empty slots,total slots,station_id,coordinates
0,38.753590,-9.157170,2025-07-26T23:39:44.571202+00:00Z,0,32,32,0,"38.75359,-9.15717"
1,38.752310,-9.158850,2025-07-26T23:39:44.571444+00:00Z,0,23,23,1,"38.75231,-9.15885"
2,38.770657,-9.160248,2025-07-26T23:39:44.570809+00:00Z,7,26,33,2,"38.770657,-9.160248"
3,38.780690,-9.096220,2025-07-26T23:39:44.570706+00:00Z,8,6,14,3,"38.78069,-9.09622"
4,38.702936,-9.175234,2025-07-26T23:39:44.571010+00:00Z,6,17,23,4,"38.702936,-9.175234"


In [7]:
# Create Parameters
ll = lb_citybikes['coordinates']

# Assign Parameters
paramters = {
    'll' : ll,
    'radius':1000,
    'fields':'name,fsq_place_id,distance,categories,attributes,price,popularity,rating,stats,hours,hours_popular,veracity_rating',
    'limit':50,
}
# Assign API URL
url = 'https://places-api.foursquare.com/places/search'

# Assign required headers
headers = {
    'accept': 'application/json',
    'authorization': 'Bearer'+api_key,
    'X-Places-Api-Version': '2025-06-17'
}

ll


0        38.75359,-9.15717
1        38.75231,-9.15885
2      38.770657,-9.160248
3        38.78069,-9.09622
4      38.702936,-9.175234
              ...         
190    38.729724,-9.157531
191    38.711498,-9.194361
192    38.739395,-9.162447
193     38.70878,-9.137125
194      38.74095,-9.13409
Name: coordinates, Length: 195, dtype: object

In [8]:
# FOURSQUARE Places API
responses = []

# Iterate through each station by its coordinates
for coordinates in ll:

    paramters = {
    'll' : coordinates,
    'radius':1000,
    'fields':'name,fsq_place_id,distance,categories,attributes,price,popularity,rating,stats,hours,hours_popular,veracity_rating',
    'limit':50,
    }
    
    response = requests.get(url,headers=headers,params=paramters)
    data = response.json()
    responses.append(data)

In [9]:
# import pprint

# # View FOURSQUARE API Responses
# pprint.pprint(responses)
responses[0]

{'results': [{'fsq_place_id': '4cf949b7feec6dcb1d0f3836',
   'categories': [{'fsq_category_id': '4bf58dd8d48988d126941735',
     'name': 'Government Building',
     'short_name': 'Government',
     'plural_name': 'Government Buildings',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_',
      'suffix': '.png'}},
    {'fsq_category_id': '4bf58dd8d48988d12f941735',
     'name': 'Library',
     'short_name': 'Library',
     'plural_name': 'Libraries',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/library_',
      'suffix': '.png'}}],
   'distance': 124,
   'attributes': {},
   'hours': {'is_local_holiday': False},
   'hours_popular': [{'close': '2000', 'day': 1, 'open': '0800'},
    {'close': '2300', 'day': 1, 'open': '2200'},
    {'close': '1900', 'day': 2, 'open': '0900'},
    {'close': '1800', 'day': 3, 'open': '0800'},
    {'close': '1800', 'day': 4, 'open': '0900'},
    {'close': '1700', 'day': 5, 'open': '0900'},
    {'c

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [10]:
# Create empty lists 
popularity_list = []
rating_list = []
total_rating_list = []
veracity_rating_list = []
distance_list = []
name_list = []
category_name_list = []
hours_list = []
coordinates_list = []

# Iterate through each station_coordinates results
for stations in responses:
    context = stations.get('context', {})
    center = context.get('geo_bounds', {}).get('circle', {}).get('center', {})
    latitute = center.get('latitude', 'N/A')
    longitude = center.get('longitude', 'N/A')
    
    results = stations.get('results','N/A')
    # Iterate through each station's results for data on Points_Of_Interest
    for station in results:
        name = station.get('name','N/A')
        rating = station.get('rating','N/A')
        popularity = station.get('popularity','N/A')
        distance = station.get('distance','N/A')
        veracity_rating = station.get('veracity_rating','N/A')

        stats = station.get('stats',{})
        total_ratings = stats.get('total_ratings',{})

        categories = station.get('categories', [])
        category_name = ', '.join([cat.get('name', 'N/A') for cat in categories])

        # Append each Point_Of_Interest to empty lists
        popularity_list.append(popularity)
        rating_list.append(rating)
        total_rating_list.append(total_ratings)
        veracity_rating_list.append(veracity_rating)
        distance_list.append(distance)
        name_list.append(name)
        category_name_list.append(category_name)
        coordinates_list.append(f"{latitute},{longitude}")

        


    



In [11]:
# Create dictionary of iterated list entires
station_POI_dic = {
    'name':name_list,
    'rating':rating_list,
    'total num ratings':total_rating_list,
    'popularity':popularity_list,
    'distance':distance_list,
    'veracity':veracity_rating_list,
    'category': category_name_list,
    'coordinates': coordinates_list

}

# Create DataFrame from Dictionary
station_POI_df = pd.DataFrame(station_POI_dic)

station_POI_df.shape


(9662, 8)

Put your parsed results into a DataFrame

In [12]:
# Assess Datframe Success
station_POI_df.head()

,name,rating,total num ratings,popularity,distance,veracity,category,coordinates
0,Torre Do Tombo,8.4,21,0.943785,124,4,"Government Building, Library","38.75359,-9.15717"
1,Aula Magna,8.5,127,0.986900,138,4,Concert Hall,"38.75359,-9.15717"
2,Alameda da Universidade,7.9,47,0.937355,121,4,"Plaza, Garden","38.75359,-9.15717"
3,Letras Bar,6.2,11,0.659332,87,3,Portuguese Restaurant,"38.75359,-9.15717"
4,Horto do Campo Grande,8.2,40,0.959301,337,5,Flower Store,"38.75359,-9.15717"


In [13]:
# Assign station_id in station_POI dataframe
station_POI_df = station_POI_df.merge(
    lb_citybikes[['coordinates', 'station_id']],
    on='coordinates',
    how='left'
)

# Remove station coorindates to minimze data redunancy and increase readability

station_POI_df.drop(columns=['coordinates'], inplace=True)





In [14]:
station_POI_df.value_counts('station_id')

station_id
0      50
130    50
121    50
122    50
123    50
       ..
139    39
191    39
182    35
173    33
132    30
Name: count, Length: 195, dtype: int64

In [15]:
# View new formatted Dataframe
station_POI_df.head()

,name,rating,total num ratings,popularity,distance,veracity,category,station_id
0,Torre Do Tombo,8.4,21,0.943785,124,4,"Government Building, Library",0
1,Aula Magna,8.5,127,0.986900,138,4,Concert Hall,0
2,Alameda da Universidade,7.9,47,0.937355,121,4,"Plaza, Garden",0
3,Letras Bar,6.2,11,0.659332,87,3,Portuguese Restaurant,0
4,Horto do Campo Grande,8.2,40,0.959301,337,5,Flower Store,0


In [16]:
# Question 3 
    # Query for Restaurants / Bars.
# Identify category types
station_POI_df.value_counts('category').head(25)

category
Portuguese Restaurant              945
Restaurant                         577
Café                               495
Bakery                             303
Coffee Shop                        171
Burger Joint                       169
Plaza                              165
Pizzeria                           158
Bar                                146
Italian Restaurant                 144
Supermarket                        133
Ice Cream Parlor                   127
Indian Restaurant                  124
Seafood Restaurant                 100
Vegan and Vegetarian Restaurant     99
Chinese Restaurant                  86
Garden                              82
BBQ Joint                           82
Park                                82
Sushi Restaurant                    77
Pharmacy                            71
Bookstore                           71
Tapas Restaurant                    68
Scenic Lookout                      68
Mediterranean Restaurant            64
Name: count, dty

In [17]:
# AI Assistance to create masks & series

# Filter masks for categories values of 'Restaurant' and 'Bar'
restaurant_mask = station_POI_df['category'].str.contains('Restaurant', case=False, na=False)
bar_mask = station_POI_df['category'].str.contains('Bar', case=False, na=False)

# Create Series for each mask-filter
restaurant_counts = station_POI_df[restaurant_mask].groupby('station_id').size().rename('restaurant_count')
bar_counts = station_POI_df[bar_mask].groupby('station_id').size().rename('bar_count')

# Concatenate Series into Dataframe 
categories_counts = pd.concat([restaurant_counts, bar_counts], axis=1).fillna(0).astype(int)
categories_counts.reset_index(inplace=True)

# Observe category values for each bike_station
categories_counts.sort_values('restaurant_count', ascending=False)


,station_id,restaurant_count,bar_count
94,94,32,0
69,69,31,4
9,9,30,3
181,181,30,1
154,154,30,4
...,...,...,...
44,44,8,1
74,74,8,0
139,139,7,0
173,173,6,0


In [18]:
# Download CSV File
station_POI_df.to_csv('FOURSQUARE_Lisbon_Station_Surroundings.csv', index=False, header=True, sep=',')

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [19]:
# Yelp](https://docs.developer.yelp.com/docs/fusion-intro) 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating